In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/LLM-with-RAG/notebook


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    PyPDFLoader,
    DirectoryLoader,
    UnstructuredHTMLLoader,
    UnstructuredWordDocumentLoader,
    BSHTMLLoader,
    Docx2txtLoader,
    TextLoader,
    SeleniumURLLoader,
    YoutubeLoader,
    UnstructuredURLLoader,
)
from unstructured.cleaners.core import clean_extra_whitespace
from langchain.text_splitter import TokenTextSplitter


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import os
import torch
from src.utils import *

In [3]:
# loader = Docx2txtLoader("web_data/sample4.docx")
# data = loader.load()
# print(data)

In [4]:
# urls = [
# "https://e.vnexpress.net/news/travel-guide/binh-phuoc-boasts-jungle-treks-charming-feasts-4673392.html",
# 'https://e.vnexpress.net/news/culture/vietnamese-comic-artist-wins-silver-at-int-l-manga-awards-4695364.html',
# ]
# loader = UnstructuredURLLoader(urls=urls, 
#                     post_processors=[clean_extra_whitespace],)
# data = loader.load()
# print(data)
#

In [5]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=TsVZJbnnaSs", add_video_info=False
)
data = loader.load()
print(data)

[Document(page_content="what is this that is not prime numbers I know 49 is not a prime number I know that 9 is not a prime number yet llama thinks that this function actually finds prime numbers accuracy aside you might want to run llama on your system locally without all the restrictions that you're gonna get to put all these Services popping up possibly charging money because you can actually take the Llama models run it and charge people for it if you want to at least that's my understanding of the terms of service unless you have over 700 million unless you're Apple or Google okay how do we run llama locally on an Apple silicon machine that's what I'm going to show you today if you don't have an apple silicon machine running llamas easy you go to the GitHub URL then I'm gonna leave all the links down below including the instructions and the steps you're gonna go to this one Facebook research slash Lama on GitHub you're going to clone this repository follow the instructions and boo

In [6]:
from transformers import pipeline
class NewsSummarizer:
    def __init__(self, summarizer = pipeline("summarization", model="Falconsai/text_summarization"),
                 max_length:int=230, 
                 min_length:int=30,
                 ):
        self.summarizer = summarizer
        self.max_length = max_length
        self.min_length = min_length
        
    def summary_text(self,text:str)->str:
        '''Summary short text'''
        text = self.summarizer(text, max_length=self.max_length, min_length=self.min_length, do_sample=False)[0]['summary_text']
        return text
    
    def summary_news(self, news:str)->str:
        '''
        Summary news. Because the news is too long (Loss of Information) so I will split news into 2 parts
        '''
        sample = news.split('. ')
        art1 = '. '.join(sample[:int(len(sample)/2)])
        art2 = '. '.join(sample[int(len(sample)/2):])
        sum_art1 = self.summary_text(art1)
        sum_art2 = self.summary_text(art2)
        summary_text = f'   {sum_art1}\n    {sum_art2}'
        return summary_text

/root/code_Bao/LLM-with-RAG/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
text = data[0].page_content
print(text)

what is this that is not prime numbers I know 49 is not a prime number I know that 9 is not a prime number yet llama thinks that this function actually finds prime numbers accuracy aside you might want to run llama on your system locally without all the restrictions that you're gonna get to put all these Services popping up possibly charging money because you can actually take the Llama models run it and charge people for it if you want to at least that's my understanding of the terms of service unless you have over 700 million unless you're Apple or Google okay how do we run llama locally on an Apple silicon machine that's what I'm going to show you today if you don't have an apple silicon machine running llamas easy you go to the GitHub URL then I'm gonna leave all the links down below including the instructions and the steps you're gonna go to this one Facebook research slash Lama on GitHub you're going to clone this repository follow the instructions and boom you're done that's if 

In [14]:
text_splitter = TokenTextSplitter(chunk_size=230, chunk_overlap=10)
texts = text_splitter.split_text(text)
print(texts[0])

what is this that is not prime numbers I know 49 is not a prime number I know that 9 is not a prime number yet llama thinks that this function actually finds prime numbers accuracy aside you might want to run llama on your system locally without all the restrictions that you're gonna get to put all these Services popping up possibly charging money because you can actually take the Llama models run it and charge people for it if you want to at least that's my understanding of the terms of service unless you have over 700 million unless you're Apple or Google okay how do we run llama locally on an Apple silicon machine that's what I'm going to show you today if you don't have an apple silicon machine running llamas easy you go to the GitHub URL then I'm gonna leave all the links down below including the instructions and the steps you're gonna go to this one Facebook research slash Lama on GitHub you're going to clone this repository follow the instructions and boom you're done that's if 

In [15]:
texts

["what is this that is not prime numbers I know 49 is not a prime number I know that 9 is not a prime number yet llama thinks that this function actually finds prime numbers accuracy aside you might want to run llama on your system locally without all the restrictions that you're gonna get to put all these Services popping up possibly charging money because you can actually take the Llama models run it and charge people for it if you want to at least that's my understanding of the terms of service unless you have over 700 million unless you're Apple or Google okay how do we run llama locally on an Apple silicon machine that's what I'm going to show you today if you don't have an apple silicon machine running llamas easy you go to the GitHub URL then I'm gonna leave all the links down below including the instructions and the steps you're gonna go to this one Facebook research slash Lama on GitHub you're going to clone this repository follow the instructions and boom you're done that's i

In [11]:
sum_doc = []
for txt in texts:
    sum_text = NewsSummarizer().summary_news(txt)
    sum_doc.append(sum_text)
    
sum_doc_text = '\n'.join(sum_doc)
print(sum_doc_text)


Your max_length is set to 230, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 230, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 230, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 230, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_len

   : . :: ! !! .. .! ? !. & ! # !
    llama thinks that this function actually finds prime numbers accuracy aside you might want to run llamas on your system locally without all the restrictions that you're gonna get to put all these Services popping up possibly charging money because you can take the Llama models run it and charge people for it if you want to at least that's my understanding of the terms of service unless you have over 700 million . I'm gonna leave all the links down below including the instructions . and the steps you'
   : . :: ! !! .. .! ? !. & ! # !
    I'm going to git clone this repository this Facebook research llama Repository I've created a folder called LLama2 . this is the C plus Port this port is done by the same person that created whisper and how to use that on the channel before this one supports Mac OS Linux Windows Docker and so on there's mirrors for python go node Ruby c-sharp Scala the gotcha .
   : . :: ! !! .. .! ? !. & ! # !
    go back to the o

In [12]:
sum_doc_text

"   : . :: ! !! .. .! ? !. & ! # !\n    llama thinks that this function actually finds prime numbers accuracy aside you might want to run llamas on your system locally without all the restrictions that you're gonna get to put all these Services popping up possibly charging money because you can take the Llama models run it and charge people for it if you want to at least that's my understanding of the terms of service unless you have over 700 million . I'm gonna leave all the links down below including the instructions . and the steps you'\n   : . :: ! !! .. .! ? !. & ! # !\n    I'm going to git clone this repository this Facebook research llama Repository I've created a folder called LLama2 . this is the C plus Port this port is done by the same person that created whisper and how to use that on the channel before this one supports Mac OS Linux Windows Docker and so on there's mirrors for python go node Ruby c-sharp Scala the gotcha .\n   : . :: ! !! .. .! ? !. & ! # !\n    go back to

In [19]:
sum_text

"   I'm going to clone this repository this Facebook research slash Lama on GitHub . I've created a folder called llama2 and you're going to go to this other repository . this is the C plus Port this is done by the same person that created whisper and how to use that on the channel before this one supports Mac OS Linux Windows Docker . if you go back to the original repository you are gonna need to scroll down a little bit and there's a\n    I'm going to run this and it's spewing out all sorts of weird stuff some of it is in German wow yeah it even has some python code so crazy stuff crazy stuff how do we get it to do something useful if I open this up in vs code we can take a look at the code first of all and you can send one of these files to the model as an example or as a starting prompt for example this chat with bob.txt or one of the other ones in"

In [12]:
len(data)

1

In [13]:
file_path = "web_data/sample3.txt"

with open(file_path, "w") as file:
    file.write(str(sum_doc_text))

In [14]:
data[0].page_content


"what is this that is not prime numbers I know 49 is not a prime number I know that 9 is not a prime number yet llama thinks that this function actually finds prime numbers accuracy aside you might want to run llama on your system locally without all the restrictions that you're gonna get to put all these Services popping up possibly charging money because you can actually take the Llama models run it and charge people for it if you want to at least that's my understanding of the terms of service unless you have over 700 million unless you're Apple or Google okay how do we run llama locally on an Apple silicon machine that's what I'm going to show you today if you don't have an apple silicon machine running llamas easy you go to the GitHub URL then I'm gonna leave all the links down below including the instructions and the steps you're gonna go to this one Facebook research slash Lama on GitHub you're going to clone this repository follow the instructions and boom you're done that's if